In [ ]:
import nltk
import numpy
import openpyxl
import pandas
import string

from ipyfilechooser import FileChooser
from nltk.sentiment import SentimentIntensityAnalyzer as sia
from openpyxl.utils.dataframe import dataframe_to_rows as df2r

pandas.options.mode.chained_assignment = None # suppress warning

In [ ]:
nltk.download(['names', 'stopwords', 'state_union', 'averaged_perceptron_tagger', 'vader_lexicon'], quiet=True)

punct = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
n = numpy.nan

In [ ]:
medium_cut = .05 # seems to be standard in literature for VADER compound score
high_cut = .2 # guess at reasonable value
adjustment_value = .067 # more than three net positive/negative statements imply high cutoff

def eval_weighting(w):
    if w < -1 * high_cut:
        return 'Very negative'
    elif w < -1 * medium_cut:
        return 'Negative'
    elif w > high_cut:
        return 'Very positive'
    elif w > medium_cut:
        return 'Positive'
    else:
        return 'Neutral'

In [ ]:
def clear_punctuation(s):
    for e in s:
        if e in punct:
            s = s.replace(e, '')
    return s

In [ ]:
def clean_data(df, cols): # prepare cells for analysis
    for i in cols:
        for j in range(len(df.columns)):
            t = df.iloc[i, j]
            if isinstance(t, str):
                u = clear_punctuation(t.strip().lower())
                if u == 'na':
                    df.iloc[i, j] = ''
                else:
                    df.iloc[i, j] = u
            else:
                df.iloc[i, j] = ''
    return df

In [ ]:
def get_col_index(df, s): # find which column holds a string
    for h in list(df.columns):
        if s in h:
            return df.columns.get_loc(h)

In [ ]:
def clean_filename(s):
    f = ''.join(c for c in s if (c.isalnum() or c in '._- '))
    return f

In [ ]:
pos_words = pandas.read_table('dictionaries/green terminology.txt', header=None)
neg_words = pandas.read_table('dictionaries/red terminology.txt', header=None)

pos_list = []
pos_weight = []
for i in range(0, len(pos_words)):
    if pos_words.iloc[i, 1] != 0: # ignore entries with zero weight
        pos_list.append(pos_words.iloc[i, 0])
        pos_weight.append(pos_words.iloc[i, 1])

neg_list = []
neg_weight = []
for i in range(0, len(neg_words)):
    if neg_words.iloc[i, 1] != 0:
        neg_list.append(neg_words.iloc[i, 0])
        neg_weight.append(neg_words.iloc[i, 1])

In [ ]:
fc = FileChooser('data')
display(fc)

In [ ]:
df = pandas.read_excel(fc.selected)

cols_of_interest = [
    get_col_index(df, 'strengths of this experience'),
    get_col_index(df, 'better learning experience'),
    get_col_index(df, 'Nothing further to add')]

col_weighting = [.95, 1.025, 1.025] # slightly overweight 'negative' and additional comments 

df = clean_data(df, cols_of_interest)

In [ ]:
def combine_weightings(l, s):
    # first parameter is lexical/syntactic score
    # second parameter is sentiment score
    w = 1 * l + 0 * s
    return w

In [ ]:
sii = sia()
weights = []
evals = []

needed_matches = 1 # seek sufficient number of lexical/syntactic matches

for i in range(0, len(df)):

    w = 0

    for j in cols_of_interest:

        if isinstance(df.iloc[i, j], str):

            t = clear_punctuation(df.iloc[i, j])
            r = 0
            b = 0

            x = cols_of_interest.index(j)

            for k, p in enumerate(pos_list):
                if p in t:
                    b += 1
                    r = min(1, r + adjustment_value * pos_weight[k] * col_weighting[x])

            for l, n in enumerate(neg_list):
                if n in t:
                    b += 1
                    r = max(-1, r - adjustment_value * neg_weight[l] * col_weighting[x])

            # t = [w for w in t.split() if w.isalpha() and w.lower() not in stopwords]
            p = sii.polarity_scores(t)['compound']

            if b < needed_matches:
                r = p
            else:
                r = combine_weightings(r, p)

            w = w + r

    weights.append(w / len(cols_of_interest))
    evals.append(eval_weighting(w / len(cols_of_interest)))

In [ ]:
f = input("What is the name of the output file? ")
f = clean_filename(f)

In [ ]:
# for training data
cols_of_interest.append(get_col_index(df, 'Rating'))

In [ ]:
#for test data
cols_of_interest.append(get_col_index(df, 'Quality of Interactions during Experience AVG'))
cols_of_interest.append(get_col_index(df, 'Quality of Preceptor/Preceptor Team AVG'))
cols_of_interest.append(get_col_index(df, 'Quality of Site AVG'))

In [ ]:
df2 = df.iloc[:, cols_of_interest]

In [ ]:
# for test data
df2['Overall Quality'] = df.iloc[:, cols_of_interest[-3:]].sum(axis=1)
df2[df2['Overall Quality'].eq('')] = n

In [ ]:
df2['Semantic Value'] = weights
df2['Semantic Evaluation'] = evals

wb = openpyxl.Workbook()
ws = wb.active
rows = df2r(df2, index=False)

for i, row in enumerate(rows, 1):
    for j, val in enumerate(row, 1):
         ws.cell(row=i, column=j, value=val)

wb.save('data/' + f + '.xlsx')